# Итоговый проект. Улицы. Объекты культурного наследия.

In [3]:
# TODO собрать установку всех необходимых модулей в одном месте
!pip install geopandas
!pip install mapclassify
!pip install keplergl
!pip install osmnx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 60.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached jedi-0.19.1-py2.py3-none-any.whl (1.6 MB)
  Created wheel for keplergl: filename=keplergl-0.3.2-py2.py3-none-any.whl size=17922316 sha256=22b7bed29ad32da5dc5282821a7a8f8d12fc2bc021198ada5e8f20f638ee11ef
  Stored in directory: /root/.cache/pip/wheels/e5/e9/db/b8b8f2fcb5cb1cd7268d88959fce267bc00697cb8f50dcd0d9
Successfully built keplergl
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 kB 2.5 MB/s eta 0:00:00


In [4]:
# TODO собрать импорты всех модулей в одном месте
import geopandas as gpd
import osmnx as ox
from keplergl import KeplerGl

In [5]:
# code snippets - enable_custom_widget_manager
from google.colab import output
output.enable_custom_widget_manager()

In [6]:
TILES = "CartoDB positron"  # Название подложки для карт

# TODO указать любой район Санкт-Петербург из OSM https://wiki.openstreetmap.org/wiki/RU:%D0%A1%D0%B0%D0%BD%D0%BA%D1%82-%D0%9F%D0%B5%D1%82%D0%B5%D1%80%D0%B1%D1%83%D1%80%D0%B3/%D0%A0%D0%B0%D0%B9%D0%BE%D0%BD%D1%8B
TERRITORY_NAME = 'Калининский район, Санкт-Петербург'  # название территории для которой будут строиться слои

# TODO указать ссылку на файл из вашего github репозитория, которая начинается с https://raw.githubusercontent.com/
KGIOP_FILE_URL = "https://raw.githubusercontent.com/baltti/PythonUrban/main/geojson_layers/kgiop_objects.geojson"  # ссылка на слой с объектами культурного наследия
STREETS_FILE_URL = "https://raw.githubusercontent.com/baltti/PythonUrban/main/geojson_layers/streets.geojson"  # ссылка на слой с улицами

## Территория

### Загрузка территории из OSM (Extract)

In [71]:
# TODO загрузить geodataframe с геометрией для территории TERRITORY_NAME
territory = ox.geocode_to_gdf(TERRITORY_NAME)

territory.explore(tiles="CartoDB dark_matter")

## Улицы

### Загрузка файла с улицами (Extract)

In [75]:
# TODO отфильтровать улицы по маске геометрии территории полученной ранее
streets = gpd.read_file(STREETS_FILE_URL, mask=territory)

streets.explore(tiles=TILES)

In [78]:
streets

,name,geometry
0,проспект Металлистов,"LINESTRING (3382971.239 8393975.910, 3382946.0..."
1,улица Карпинского,"LINESTRING (3386426.663 8400785.882, 3386439.6..."
2,Брюсовская улица,"LINESTRING (3386897.867 8396690.882, 3386910.8..."
3,улица Карпинского,"LINESTRING (3386482.100 8400818.592, 3386603.4..."
4,Меншиковский проспект,"LINESTRING (3386719.834 8397463.093, 3386684.0..."
...,...,...
990,Светлановский проспект,"LINESTRING (3384444.330 8409666.286, 3384458.5..."
991,Боткинская улица,"LINESTRING (3379065.985 8390345.643, 3379084.3..."
992,Гражданский проспект,"LINESTRING (3383552.516 8402053.053, 3383508.7..."
993,аллея Академика Глушко,"LINESTRING (3380910.426 8405232.109, 3380854.5..."


### Обработка данных с улицами (Transform)

In [80]:
# TODO сгруппировать и объединить геометрии с одинаковыми названиями
streets["name"].is_unique


False

In [81]:
streets_union = streets.dissolve(by="name", as_index=False)
streets_union

,name,geometry
0,1-й Муринский проспект,"LINESTRING (3377526.737 8397304.736, 3377732.2..."
1,Амурская улица,"MULTILINESTRING ((3386247.695 8399614.950, 338..."
2,Антоновская улица,"MULTILINESTRING ((3384111.863 8393950.170, 338..."
3,Арсенальная набережная,"MULTILINESTRING ((3378798.261 8389383.864, 337..."
4,Арсенальная улица,"MULTILINESTRING ((3380194.898 8393207.178, 338..."
...,...,...
113,улица Федосеенко,"MULTILINESTRING ((3383800.380 8394638.840, 338..."
114,улица Харченко,"LINESTRING (3378249.156 8397461.001, 3378229.4..."
115,улица Хлопина,"MULTILINESTRING ((3380328.292 8400060.278, 338..."
116,улица Черкасова,"MULTILINESTRING ((3386318.739 8404988.765, 338..."


### Сохранение слоя с улицами (Load)

In [82]:
# TODO переименовать столбцы в русские названия, кроме столбца geometry
streets_union = streets_union.rename(columns={"name": "название"})

# TODO для того чтобы переименовать индекс, нужно обратиться и нему и вызвать от него метод rename (https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Index.rename.html)
streets_union.index.rename("индекс", inplace=True)

streets_union

,название,geometry
индекс,,
0,1-й Муринский проспект,"LINESTRING (3377526.737 8397304.736, 3377732.2..."
1,Амурская улица,"MULTILINESTRING ((3386247.695 8399614.950, 338..."
2,Антоновская улица,"MULTILINESTRING ((3384111.863 8393950.170, 338..."
3,Арсенальная набережная,"MULTILINESTRING ((3378798.261 8389383.864, 337..."
4,Арсенальная улица,"MULTILINESTRING ((3380194.898 8393207.178, 338..."
...,...,...
113,улица Федосеенко,"MULTILINESTRING ((3383800.380 8394638.840, 338..."
114,улица Харченко,"LINESTRING (3378249.156 8397461.001, 3378229.4..."
115,улица Хлопина,"MULTILINESTRING ((3380328.292 8400060.278, 338..."


In [83]:
# TODO сохранить слой в географической проекции в формате GeoJSON
streets_union.to_crs(4326).to_file('union_streets.geojson', driver='GeoJSON')

## Объекты культурного наследия

### Загрузка объектов культурного наследия (Extract)

In [84]:
# TODO отфильтровать улицы по маске геометрии территории полученной ранее
kgiop_objects = gpd.read_file(KGIOP_FILE_URL, mask=territory)

kgiop_objects

,id,ensemble_name,object_name,occurrence_time,object_location,historical_category,normative_act,object_type,geometry
0,2353,—,Пожарная часть,1930,"Лесной пр., 17",выявленный объект культурного наследия,Приказ председателя КГИОП № 15 от 20.02.2001,Памятник,POINT (3378699.369 8392081.702)
1,2354,—,Народный дом Нобеля,—,"Лесной пр., 19",объект культурного наследия регионального знач...,Закон Санкт-Петербурга № 141-47 от 02.07.1997,Памятник,POINT (3378678.458 8392221.495)
2,2230,Комплекс Санкт-Петербургской одиночной тюрьмы ...,Прачечная и ледник,1885-1890,"Арсенальная наб., 7; Комсомола ул., 8",выявленный объект культурного наследия,Приказ председателя КГИОП № 15 от 20.02.2001,Памятник,POINT (3379975.413 8389295.482)
3,2180,—,"Квартира, в которой жили в 1924-1934 гг. химик...",—,"Академика Лебедева ул., 10-г, кв. 4",объект культурного наследия регионального знач...,Закон Санкт-Петербурга № 174-27 от 05.07.1999,Памятник,POINT (3379068.019 8391163.126)
4,2181,—,Доходный дом Российского общества страхования ...,1912-1914,"Академика Лебедева ул., 12",выявленный объект культурного наследия,Приказ председателя КГИОП № 15 от 20.02.2001,Памятник,POINT (3379054.432 8391248.345)
...,...,...,...,...,...,...,...,...,...
264,9447,Комплекс зданий Агрофизического научно-исследо...,Главное здание,—,"Гражданский пр., 14, лит. А",выявленный объект культурного наследия,Распоряжение КГИОП от 31.05.2016 № 10-218,Памятник,POINT (3382321.835 8399879.935)
265,9448,Комплекс зданий Агрофизического научно-исследо...,Лаборатория свето-физиологии с оранжереей,—,"Гражданский пр., 14, лит. Д",выявленный объект культурного наследия,Распоряжение КГИОП от 31.05.2016 № 10-218,Памятник,POINT (3382290.777 8399731.656)
266,9534,Кондратьевский жилмассив,Жилые здания,1929-1931,"Санкт-Петербург , Кондратьевский проспект, 40",объект культурного наследия регионального знач...,Распоряжение КГИОП № 10-89 от 27.02.2014,Памятник,POINT (3382123.686 8392992.569)
267,9535,Кондратьевский жилмассив,Здание Калининского универмага,1929-1931,"Санкт-Петербург , Кондратьевский проспект, 40",объект культурного наследия регионального знач...,Распоряжение КГИОП № 10-89 от 27.02.2014,Памятник,POINT (3382476.012 8393048.401)


### Обработка объектов культурного наследия (Transform)

In [85]:
# TODO добавить два столбца lon и lat, в которых будут долгота и широта
kgiop_objects["lon"] = kgiop_objects.to_crs(3857).geometry.centroid.to_crs(4326).x
kgiop_objects["lat"] = kgiop_objects.to_crs(3857).geometry.centroid.to_crs(4326).y

kgiop_objects

,id,ensemble_name,object_name,occurrence_time,object_location,historical_category,normative_act,object_type,geometry,lon,lat
0,2353,—,Пожарная часть,1930,"Лесной пр., 17",выявленный объект культурного наследия,Приказ председателя КГИОП № 15 от 20.02.2001,Памятник,POINT (3378699.369 8392081.702),30.351373,59.965594
1,2354,—,Народный дом Нобеля,—,"Лесной пр., 19",объект культурного наследия регионального знач...,Закон Санкт-Петербурга № 141-47 от 02.07.1997,Памятник,POINT (3378678.458 8392221.495),30.351185,59.966222
2,2230,Комплекс Санкт-Петербургской одиночной тюрьмы ...,Прачечная и ледник,1885-1890,"Арсенальная наб., 7; Комсомола ул., 8",выявленный объект культурного наследия,Приказ председателя КГИОП № 15 от 20.02.2001,Памятник,POINT (3379975.413 8389295.482),30.362836,59.953064
3,2180,—,"Квартира, в которой жили в 1924-1934 гг. химик...",—,"Академика Лебедева ул., 10-г, кв. 4",объект культурного наследия регионального знач...,Закон Санкт-Петербурга № 174-27 от 05.07.1999,Памятник,POINT (3379068.019 8391163.126),30.354684,59.961463
4,2181,—,Доходный дом Российского общества страхования ...,1912-1914,"Академика Лебедева ул., 12",выявленный объект культурного наследия,Приказ председателя КГИОП № 15 от 20.02.2001,Памятник,POINT (3379054.432 8391248.345),30.354562,59.961847
...,...,...,...,...,...,...,...,...,...,...,...
264,9447,Комплекс зданий Агрофизического научно-исследо...,Главное здание,—,"Гражданский пр., 14, лит. А",выявленный объект культурного наследия,Распоряжение КГИОП от 31.05.2016 № 10-218,Памятник,POINT (3382321.835 8399879.935),30.383914,60.000638
265,9448,Комплекс зданий Агрофизического научно-исследо...,Лаборатория свето-физиологии с оранжереей,—,"Гражданский пр., 14, лит. Д",выявленный объект культурного наследия,Распоряжение КГИОП от 31.05.2016 № 10-218,Памятник,POINT (3382290.777 8399731.656),30.383635,59.999972
266,9534,Кондратьевский жилмассив,Жилые здания,1929-1931,"Санкт-Петербург , Кондратьевский проспект, 40",объект культурного наследия регионального знач...,Распоряжение КГИОП № 10-89 от 27.02.2014,Памятник,POINT (3382123.686 8392992.569),30.382134,59.969689
267,9535,Кондратьевский жилмассив,Здание Калининского универмага,1929-1931,"Санкт-Петербург , Кондратьевский проспект, 40",объект культурного наследия регионального знач...,Распоряжение КГИОП № 10-89 от 27.02.2014,Памятник,POINT (3382476.012 8393048.401),30.385299,59.969940


In [90]:
kgiop_objects.explore(tiles=TILES)

### Сохранение слоя с объектами культурного наследия (Load)

In [86]:
kgiop_objects.columns.values.tolist()

['id',
 'ensemble_name',
 'object_name',
 'occurrence_time',
 'object_location',
 'historical_category',
 'normative_act',
 'object_type',
 'geometry',
 'lon',
 'lat']

In [87]:
# TODO переименовать столбцы в русские названия, кроме столбца geometry
kgiop_objects = kgiop_objects.rename(columns={"id": "порядковый номер",
                                              'ensemble_name': "название архитектурного ансамбля",
                                              'object_name':'название объекта',
                                              'occurrence_time':"период строительства",
                                              'object_location':"адрес",
                                              'historical_category':"статус объекта",
                                              'normative_act':"основание для установления статуса",
                                              'object_type':"тип объекта",
                                              'lon':"долгота",
                                              'lat':"широта"})

# TODO для того чтобы переименовать индекс, нужно обратиться и нему и вызвать от него метод rename (https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Index.rename.html)
kgiop_objects.index.rename("индекс", inplace=True)

kgiop_objects

,порядковый номер,название архитектурного ансамбля,название объекта,период строительства,адрес,статус объекта,основание для установления статуса,тип объекта,geometry,долгота,широта
индекс,,,,,,,,,,,
0,2353,—,Пожарная часть,1930,"Лесной пр., 17",выявленный объект культурного наследия,Приказ председателя КГИОП № 15 от 20.02.2001,Памятник,POINT (3378699.369 8392081.702),30.351373,59.965594
1,2354,—,Народный дом Нобеля,—,"Лесной пр., 19",объект культурного наследия регионального знач...,Закон Санкт-Петербурга № 141-47 от 02.07.1997,Памятник,POINT (3378678.458 8392221.495),30.351185,59.966222
2,2230,Комплекс Санкт-Петербургской одиночной тюрьмы ...,Прачечная и ледник,1885-1890,"Арсенальная наб., 7; Комсомола ул., 8",выявленный объект культурного наследия,Приказ председателя КГИОП № 15 от 20.02.2001,Памятник,POINT (3379975.413 8389295.482),30.362836,59.953064
3,2180,—,"Квартира, в которой жили в 1924-1934 гг. химик...",—,"Академика Лебедева ул., 10-г, кв. 4",объект культурного наследия регионального знач...,Закон Санкт-Петербурга № 174-27 от 05.07.1999,Памятник,POINT (3379068.019 8391163.126),30.354684,59.961463
4,2181,—,Доходный дом Российского общества страхования ...,1912-1914,"Академика Лебедева ул., 12",выявленный объект культурного наследия,Приказ председателя КГИОП № 15 от 20.02.2001,Памятник,POINT (3379054.432 8391248.345),30.354562,59.961847
...,...,...,...,...,...,...,...,...,...,...,...
264,9447,Комплекс зданий Агрофизического научно-исследо...,Главное здание,—,"Гражданский пр., 14, лит. А",выявленный объект культурного наследия,Распоряжение КГИОП от 31.05.2016 № 10-218,Памятник,POINT (3382321.835 8399879.935),30.383914,60.000638
265,9448,Комплекс зданий Агрофизического научно-исследо...,Лаборатория свето-физиологии с оранжереей,—,"Гражданский пр., 14, лит. Д",выявленный объект культурного наследия,Распоряжение КГИОП от 31.05.2016 № 10-218,Памятник,POINT (3382290.777 8399731.656),30.383635,59.999972
266,9534,Кондратьевский жилмассив,Жилые здания,1929-1931,"Санкт-Петербург , Кондратьевский проспект, 40",объект культурного наследия регионального знач...,Распоряжение КГИОП № 10-89 от 27.02.2014,Памятник,POINT (3382123.686 8392992.569),30.382134,59.969689


In [88]:
# TODO сохранить слой в географической проекции в формате GeoJSON
kgiop_objects.to_crs(4326).to_file('kgiop_objects.geojson', driver='GeoJSON')